Just testing...

In [ ]:

# # input/output dimension of system
# const ndim = 4

# # width of hidden layers
# const ld = 4

# # learning rate
# const η = .001

# # number of training runs
# const runs = 1000

# #evaluate neural network
# function network(x, model)
# 	# input layer
# 	layer0 = tanh.(model[1].W * x)

# 	# first hidden layer
# 	layer1 = tanh.(model[2].W * layer0 .+ model[2].b)
	
# 	# second hidden layer
# 	layer2 = tanh.(model[3].W * layer1 .+ model[3].b)

# 	# output layer (linear activation)
# 	model[4].W * layer2
# end

# model = (
# 	(W = randn(ld, ndim), ),
# 	(W = randn(ld, ld), b = randn(ld)),
# 	(W = randn(ld, ld), b = randn(ld)),
# 	(W = randn(ndim, ld), ),
# )

In [ ]:
# x = [1, 2, 3, 4]
# network(x, model)

Code from Sparsification module

Check number of threads being used first

In [ ]:
num_threads = Threads.nthreads()

In [ ]:
using Flux

In [ ]:
using Distributions
using GeometricIntegrators
using Optim
using Random
using Distances
using Symbolics
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

_prod(a, b, c, arrs...) = a .* _prod(b, c, arrs...)
_prod(a, b) = a .* b
_prod(a) = a

# generates a vector out of symbolic arrays (p,q) with a certain dimension
function get_z_vector(dims)
    @variables q[1:dims]
    @variables p[1:dims]
    z = vcat(q,p)
    return z
end

# make combinations of bases of just the order that is given 
# e.g order = 2 will give just the bases whose powers sum to 2
function poly_combos(z, order, inds...)
    if order == 0
        return Num[1]
    elseif order == length(inds)
        return [_prod([z[i] for i in inds]...)]
    else
        start_ind = length(inds) == 0 ? 1 : inds[end]
        return vcat([poly_combos(z, order, inds..., j) for j in start_ind:length(z)]...)
    end
end

# gives all bases monomials up to a certain order
function primal_monomial_basis(z, order::Int)
    return Vector{Symbolics.Num}(vcat([poly_combos(z, i) for i in 1:order]...))
end

# calculates coefficient bases up to a certain order
# mostly for use with trigonometric functions example sin(k*z),
# where k is the coefficient
function primal_coeff_basis(z, max_coeff::Int)
    return Vector{Symbolics.Num}(vcat([k .* z for k in 1:max_coeff]...))
end

# calculates +,-,*,/ between states as a new basis
# the return output is a set to avoid duplicates
function primal_operator_basis(z, operator)
    return Vector{Symbolics.Num}([operator(z[i], z[j]) for i in 1:length(z)-1 for j in i+1:length(z)] ∪ [operator(z[j], z[i]) for i in 1:length(z)-1 for j in i+1:length(z)])
end

function primal_power_basis(z, max_power::Int)
    if max_power > 0
        return Vector{Symbolics.Num}(vcat([z.^i for i in 1:max_power]...))
    elseif max_power < 0
        return Vector{Symbolics.Num}(vcat([z.^-i for i in 1:abs(max_power)]...))
    end
end

function polynomial_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = primal_monomial_basis(z, polyorder)
    primes = vcat(primes, primal_coeff_basis(z, max_coeff))
    if operator !== nothing
        primes = vcat(primes, primal_operator_basis(z, operator))
    end
    return primes
end

function trigonometric_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return vcat(sin.(primes), cos.(primes))
end

function exponential_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return exp.(primes)
end

function logarithmic_basis(z::Vector{Symbolics.Num} = get_z_vector(2); polyorder::Int = 0, operator=nothing, max_coeff::Int = 0)
    primes = polynomial_basis(z, polyorder = polyorder, operator = operator, max_coeff = max_coeff)
    return log.(abs.(primes))
end

function mixed_states_basis(basis::Vector{Symbolics.Num}...)
    mixed_states = Tuple(basis)
    
    ham = Vector{Symbolics.Num}()
    for i in eachindex(mixed_states)
        for j in i+1:lastindex(mixed_states)
            ham = vcat(ham, [mixed_states[i][k] * mixed_states[j][l] for k in 1:length(mixed_states[i]) for l in 1:length(mixed_states[j])])
        end
    end
    
    return Vector{Symbolics.Num}(ham)
end

# returns the number of required coefficients for the basis
function get_numCoeffs(basis::Vector{Symbolics.Num})
    return length(basis)
end


# gets a vector of combinations of hamiltonian basis
function get_basis_set(basis::Vector{Symbolics.Num}...)
    # gets a vector of combinations of basis
    basis = vcat(basis...)
    
    # removes duplicates
    basis = Vector{Symbolics.Num}(collect(unique(basis)))

    return basis
end

# returns a function that can build the gradient of the hamiltonian
function ΔH_func_builder(d::Int, z::Vector{Symbolics.Num} = get_z_vector(d), basis::Vector{Symbolics.Num}...) 
    # nd is the total number of dimensions of all the states, e.g. if q,p each of 3 dims, that is 6 dims in total
    nd = 2d
    Dz = Differential.(z)
    
    # collects and sums combinations of basis and coefficients"
    basis = get_basis_set(basis...)
   
    # gets number of terms in the basis
    @variables a[1:get_numCoeffs(basis)]
    
    # collect and sum combinations of basis and coefficients
    ham = sum(collect(a .* basis))
    
    # gives derivative of the hamiltonian, but not the skew-symmetric true one
    f = [expand_derivatives(dz(ham)) for dz in Dz]

    #simplify the expression potentially to make it faster
    f = simplify(f)
    
    # line below makes the vector into a hamiltonian vector field by multiplying with the skew-symmetric matrix
    ∇H = vcat(f[d+1:2d], -f[1:d])
    
    # builds a function that calculates Hamiltonian gradient and converts the function to a native Julia function
    ∇H_eval = @RuntimeGeneratedFunction(Symbolics.inject_registered_module_functions(build_function(∇H, z, a)[2]))
    
    return ∇H_eval
end

struct HamiltonianSINDy{T, GHT}
    basis::Vector{Symbolics.Num} # the augmented basis for sparsification
    analytical_fθ::GHT
    z::Vector{Symbolics.Num} 
    λ::T # Sparsification Parameter
    noise_level::T # Noise amplitude added to the data
    noiseGen_timeStep::T # Time step for the integrator to get noisy data 
    nloops::Int # Sparsification Loops
    
    function HamiltonianSINDy(basis::Vector{Symbolics.Num},
        analytical_fθ::GHT = missing,
        z::Vector{Symbolics.Num} = get_z_vector(2);
        λ::T = 0.05,
        noise_level::T = 0.00,
        noiseGen_timeStep::T = 0.1,
        nloops = 10) where {T, GHT <: Union{Base.Callable,Missing}}

        new{T, GHT}(basis, analytical_fθ, z, λ, noise_level, noiseGen_timeStep, nloops)
    end
end

function gen_noisy_ref_data(method::HamiltonianSINDy, x)
    # initialize timestep data for analytical solution
    tstep = method.noiseGen_timeStep
    tspan = (zero(tstep), tstep)

    function next_timestep(x)
        prob_ref = ODEProblem((dx, t, x, params) -> method.analytical_fθ(dx, x, params, t), tspan, tstep, x)
        sol = integrate(prob_ref, Gauss(2))
        sol.q[end]
    end

    data_ref = [next_timestep(_x) for _x in x]

    # add noise
    data_ref_noisy = [_x .+ method.noise_level .* randn(size(_x)) for _x in data_ref]

    return data_ref_noisy

end

struct TrainingData{AT<:AbstractArray}
    x::AT # initial condition
    ẋ::AT # initial condition
    y::AT # noisy data at next time step

    TrainingData(x::AT, ẋ::AT, y::AT) where {AT} = new{AT}(x, ẋ, y)
    TrainingData(x::AT, ẋ::AT) where {AT} = new{AT}(x, ẋ)
end



In [ ]:
# --------------------
# Setup
# --------------------

println("Setting up...")

# 2D system with 4 variables [q₁, q₂, p₁, p₂]
const nd = 4

z = get_z_vector(nd/2)
polynomial = polynomial_basis(z, polyorder=3)
trigonometric  = trigonometric_basis(z, max_coeff=1)
prime_diff = primal_operator_basis(z, -)
basis = get_basis_set(polynomial, trigonometric, prime_diff)
# initialize analytical function, keep λ smaller than ϵ so system is identifiable
ϵ = 0.5
m = 1

# two-dim simple harmonic oscillator (not used anywhere only in case some testing needed)
# H_ana(x, p, t) = ϵ * x[1]^2 + ϵ * x[2]^2 + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2
# H_ana(x, p, t) = cos(x[1]) + cos(x[2]) + 1/(2*m) * x[3]^2 + 1/(2*m) * x[4]^2

# Gradient function of the 2D hamiltonian
# grad_H_ana(x) = [x[3]; x[4]; -2ϵ * x[1]; -2ϵ * x[2]]
grad_H_ana(x) = [x[3]; x[4]; sin(x[1]); sin(x[2])]
function grad_H_ana!(dx, x, p, t)
    dx .= grad_H_ana(x)
end
# ------------------------------------------------------------
# Training Data
# ------------------------------------------------------------

println("Generate Training Data...")

# number of samples
num_samp = 13

# samples in p and q space
samp_range = LinRange(-20, 20, num_samp)

# initialize vector of matrices to store ODE solve output

# s depend on size of nd (total dims), 4 in the case here so we use samp_range x samp_range x samp_range x samp_range
s = collect(Iterators.product(fill(samp_range, nd)...))


# compute vector field from x state values
x = [collect(s[i]) for i in eachindex(s)]
dx = zeros(nd)
p = 0
t = 0
ẋ = [grad_H_ana!(copy(dx), _x, p, t) for _x in x]


# ----------------------------------------
# Compute Sparse Regression
# ----------------------------------------

# choose SINDy method
# (λ parameter must be close to noise value so that only coeffs with value around the noise are sparsified away)
# noiseGen_timeStep chosen randomly for now
method = HamiltonianSINDy(basis, grad_H_ana!, z, λ = 0.05, noise_level = 0.00, noiseGen_timeStep = 0.1)

# generate noisy references data at next time step
y = gen_noisy_ref_data(method, x)

# collect training data
tdata = TrainingData(x, ẋ, y)

In [ ]:
# dimension of system
d = size(tdata.x[begin], 1) ÷ 2

# returns function that builds hamiltonian gradient through symbolics
fθ = ΔH_func_builder(d, method.z, method.basis)

In [ ]:

# evaluate neural network
function network_two(xᵢₙ, model, method, fθ)
	# input layer
	input_layer = tanh.(model[1].W  * xᵢₙ)

	# first hidden layer
	x₀ = tanh.(model[2].W * input_layer .+ model[2].b)

	# SINDy layer
	function SINDy_layer()
        # coeffs initialized to a vector of zeros b/c easier to optimize zeros for our case
        coeffs = model[3].W

        numLoops = 4 # random choice of loop steps
        
        local x̄ = zeros(eltype(coeffs), axes(x₀))
        local x̃ = zeros(eltype(coeffs), axes(x₀))
        local f = zeros(eltype(coeffs), axes(x₀))

        # gradient at current (x) values
        fθ(f, x₀, coeffs)

        # for first guess use explicit euler
        x̃ .= x₀ .+ method.noiseGen_timeStep .* f
        
        for _ in 1:numLoops
            x̄ .= (x₀ .+ x̃) ./ 2
            # find gradient at {(x̃ₙ + x̃ⁱₙ₊₁)/2} to get Hermite extrapolation
            fθ(f, x̄, coeffs)
            # mid point rule for integration to next step
            x̃ .= x₀ .+ method.noiseGen_timeStep .* f
        end
        return x̃
    end

	# output layer (linear activation)
	return (model[4].W * SINDy_layer())
end

In [ ]:
function loss_kernel(xᵢₙ, x₁, model, method, fθ)
    x̃₁ = network_two(xᵢₙ, model, method, fθ)
    # calculate square Euclidean distance
    sqeuclidean(x̃₁, x₁)
end

In [ ]:
# define loss function
function loss(flattened_model::AbstractVector)
    # Convert the flattened parameters back to the original structure
    local recon_model = reconstruct_model(flattened_model, ld, ndim)
    mapreduce(z -> loss_kernel(z..., recon_model, method, fθ), +, zip(tdata.x, tdata.y))
end

In [ ]:
function flatten_model(model)
    θ = [model[1].W, model[2].W, model[2].b, model[3].W, model[4].W]
    # Flatten the model into a single vector
    return flattened_model = cat([vec(θ[i]) for i in 1:length(θ)]..., dims=1)
end

In [ ]:
function reconstruct_model(flattened_model,ld,ndim)
    reconstructed_model = (
        (W = reshape(flattened_model[1:ld*ndim], ld, ndim), ),
        (W = reshape(flattened_model[ld*ndim+1:ld*(ld+ndim)], ld, ld), b = reshape(flattened_model[ld*(ld+ndim)+1:ld*(ld+ndim)+ld], ld)),
        (W = flattened_model[ld*(ld+ndim)+ld+1:ld*(ld+ndim)+ld+get_numCoeffs(method.basis)], ),
        (W = reshape(flattened_model[ld*(ld+ndim)+ld+get_numCoeffs(method.basis)+1:end], ndim, ld), ),
    )
    return reconstructed_model
end

In [ ]:
# latent dimension: ld
ld = size(tdata.x[begin], 1)
ndim = size(tdata.x[begin], 1)
model = (
	(W = randn(ld, ndim), ),
	(W = randn(ld, ld), b = randn(ld)),
	(W = zeros(get_numCoeffs(method.basis)), ),
	(W = randn(ndim, ld), ),
)

In [ ]:
# initial guess
println("Initial Guess...")
flattened_model = flatten_model(model)
# Define the optimization solver
solver = BFGS()
result = Optim.optimize(loss, flattened_model, solver, Optim.Options(show_trace=true, iterations=20); autodiff = :forward)

reconstructed_model = reconstruct_model(result.minimizer, ld, ndim)

coeffs = reconstructed_model[3].W

(result)

In [ ]:

for n in 1:method.nloops
    println("Iteration #$n...")

    # coeffs = flattened_model[ld*(ld+ndim)+ld+1:end - ld*ndim]

    # find coefficients below λ threshold
    smallinds = abs.(coeffs) .< method.λ
    biginds = .~smallinds

    # check if there are any small coefficients != 0 left
    #TODO: is the code expected to exit the loop here usually?
    all(coeffs[smallinds] .== 0) && break

    # set all small coefficients to zero
    coeffs[smallinds] .= 0

    # Regress dynamics onto remaining terms to find sparse coeffs
    function sparseloss(flattened_model::AbstractVector)
        c = zeros(eltype(flattened_model), axes(coeffs))
        c[biginds] .= flattened_model[ld*(ld+ndim)+ld+1:end - ld*ndim]

        local reconstructed_model = (
            (W = reshape(flattened_model[1:ld*ndim], ld, ndim), ),
            (W = reshape(flattened_model[ld*ndim+1:ld*(ld+ndim)], ld, ld), b = reshape(flattened_model[ld*(ld+ndim)+1:ld*(ld+ndim)+ld], ld)),
            (W = c, ),
            (W = reshape(flattened_model[end - ld*ndim + 1:end], ndim, ld), ),
        )
        loss(flatten_model(reconstructed_model))
    end

    # θ is partly a reference to coeffs[biginds] so coeffs[biginds] will be updated
    θ = [reconstructed_model[1].W, reconstructed_model[2].W, reconstructed_model[2].b, coeffs[biginds], reconstructed_model[4].W]
    # Flatten the model into a single vector
    flattened_model = cat([vec(θ[i]) for i in 1:length(θ)]..., dims=1)
    
    result = Optim.optimize(sparseloss, flattened_model, solver, Optim.Options(show_trace=true, iterations=10); autodiff = :forward)
    
    coeffs[biginds] = result.minimizer[ld*(ld+ndim)+ld+1:end - ld*ndim]

    reconstructed_model = (
        (W = reshape(result.minimizer[1:ld*ndim], ld, ndim), ),
        (W = reshape(result.minimizer[ld*ndim+1:ld*(ld+ndim)], ld, ld), b = reshape(result.minimizer[ld*(ld+ndim)+1:ld*(ld+ndim)+ld], ld)),
        (W = coeffs, ),
        (W = reshape(result.minimizer[end - ld*ndim + 1:end], ndim, ld), ),
    )
    println(result)
end

return reconstructed_model

In [ ]:
coeffs_result = reconstructed_model[3].W

In [ ]:
function (vectorfield)(dz, z)
    fθ(dz, z, reconstructed_model[3].W)
    return dz
end

(vectorfield)(dz, z, p, t) = vectorfield(dz, z)


In [ ]:

# ----------------------------------------
# Plot Results
# ----------------------------------------

println("Plotting...")

tstep = 0.01
tspan = (0.0,25.0)

for i in 1:5
    idx = rand(1:length(s))

    prob_reference = ODEProblem((dx, t, x, params) -> grad_H_ana!(dx, x, params, t), tspan, tstep, x[idx])
    data_reference = integrate(prob_reference, Gauss(2))

    prob_sindy = ODEProblem((dx, t, x, params) -> vectorfield(dx, x, params, t), tspan, tstep, x[idx])
    data_sindy = integrate(prob_sindy, Gauss(2))

    p1 = plot(xlabel = "Time", ylabel = "q₁")
    scatter!(p1, data_reference.t, data_reference.q[:,1], label = "Data q₁")
    scatter!(p1, data_sindy.t, data_sindy.q[:,1], markershape=:xcross, label = "Identified q₁")

    p3 = plot(xlabel = "Time", ylabel = "p₁")
    scatter!(p3, data_reference.t, data_reference.q[:,3], label = "Data p₁")
    scatter!(p3, data_sindy.t, data_sindy.q[:,3], markershape=:xcross, label = "Identified p₁")

    plot!(size=(1000,1000))
    display(plot(p1, p3, title="Analytical vs Calculated q₁ & p₁ in a 2D system with Euler"))

    p2 = plot(xlabel = "Time", ylabel = "q₂")
    scatter!(p2, data_reference.t, data_reference.q[:,2], label = "Data q₂")
    scatter!(p2, data_sindy.t, data_sindy.q[:,2], markershape=:xcross, label = "Identified q₂")

    p4 = plot(xlabel = "Time", ylabel = "p₂")
    scatter!(p4, data_reference.t, data_reference.q[:,4], label = "Data p₂")
    scatter!(p4, data_sindy.t, data_sindy.q[:,4], markershape=:xcross, label = "Identified p₂")

    plot!(size=(1000,1000))
    display(plot(p2, p4, title="Analytical vs Calculated q₂ & p₂ in a 2D system with Euler"))

end